# CS 484 :: Data Mining :: George Mason University :: Spring 2025


# Homework 4: Association Rule Mining

- **100 points [6% of your final grade]**
- **Due Monday, May 7 by 11:59pm**

- *Goals of this homework:* implement the association rule mining process with the Apriori algorithm.

- *Submission instructions:* for this homework, you only need to submit to Canvas. Please name your submission **FirstName_Lastname_hw4.ipynb**, so for example, my submission would be something like **Ziwei_Zhu_hw4.ipynb**. Your notebook should be fully executed so that we can see all outputs. 

In this assignment, you are going to examine movies using our understanding of association rules. For this part, you need to implement the apriori algorithm, and apply it to a movie rating dataset to find association rules of user-rate-movie behaviors. First, run the next cell to load the dataset we are going to use.

In [13]:
import numpy as np
user_movie_data = np.loadtxt("movie_rated.txt", delimiter=',')
print('array of user-movie matrix: shape ' + str(np.shape(user_movie_data)))

array of user-movie matrix: shape (11743, 2)


In this dataset, there are two columns: the first column is the integer ids of users, and the second column is the integer ids of movies. Each row denotes that the user of given user id watched the movie of the given movie id. We are going to treat each user as a transaction, so you will need to collect all the movies that have been watched by a single user as a transaction. 

Now, you need to implement the apriori algorithm and apply it to this dataset to find association rules of user movie-watching behaviors with **minimum support of 0.2** and **minimum confidence of 0.8**. We know there are many existing implementations of apriori online (check github for some good starting points). You are welcome to read existing codebases and let that inform your approach. 

**Note: Do not copy-paste any existing code.**

**Note: We want your code to have sufficient comments to explain your steps, to show us that you really know what you are doing.**

**Note: You should add print statements to print out the intermediate steps of your method -- e.g., the size of the candidate set at each step of the method, the size of the filtered set, and any other important information you think will explain the process of the method.**

**Hint: If you implement your algorithm correctly, you should be able to see intermediate result as:**
- Candidates of length 1 count: 408
- After Pruning count: 21
- Candidates of length 2 count: 210
- After Pruning 2 count: 36
- Candidates of length 3 count: 55
- After Pruning 3 count: 12
- Candidates of length 4 count: 1
- After Pruning 4 count: 0

**Hint: "Candidates of length 1/2/3/4 count" can be different, depending on what methods you use to generate candidate sets. But, your "After Pruning count" should be the same as what is shown above.**

In [20]:
# Write your code
#include all the helpful print statements
# when you run this block, we want to see all of your intermediate steps
# you can save the rules you discover for printing in the following cells (this will help us grade by keeping these separate)
from itertools import combinations
from collections import defaultdict
import pandas as pd


def get_frequent_itemsets(transactions, candidates, min_support, total_transactions):
    """Filter candidate itemsets based on min support."""
    itemset_count = defaultdict(int)
    
    # Count how many transactions contain each candidate itemset
    for transaction in transactions:
        for itemset in candidates:
            if itemset.issubset(transaction):
                itemset_count[itemset] += 1
    
    # Compute support and filter
    frequent_itemsets = {}
    for itemset, count in itemset_count.items():
        support = count / total_transactions
        if support >= min_support:
            frequent_itemsets[itemset] = support
    
    return frequent_itemsets

def generate_new_candidates(prev_frequent_itemsets, k):
    """Generate candidate itemsets of length k from (k-1)-itemsets."""
    """Generate candidate itemsets of length k from (k-1)-itemsets with pruning."""
    candidates = set()
    prev_itemsets = list(prev_frequent_itemsets)
    
    for i in range(len(prev_itemsets)):
        for j in range(i + 1, len(prev_itemsets)):
            l1 = sorted(list(prev_itemsets[i]))
            l2 = sorted(list(prev_itemsets[j]))
            if l1[:k-2] == l2[:k-2]:  # join step
                union = prev_itemsets[i].union(prev_itemsets[j])
                if len(union) == k:
                    # prune step
                    subsets = combinations(union, k - 1)
                    if all(frozenset(s) in prev_frequent_itemsets for s in subsets):
                        candidates.add(union)
    return candidates

def generate_association_rules(frequent_itemsets, min_confidence, support_data):
    """Generate association rules from frequent itemsets."""
    rules = []
    for itemset in frequent_itemsets:
        if len(itemset) < 2:
            continue
        for i in range(1, len(itemset)):
            for antecedent in combinations(itemset, i):
                antecedent = frozenset(antecedent)
                consequent = itemset - antecedent
                if consequent:
                    confidence = support_data[itemset] / support_data[antecedent]
                    if confidence >= min_confidence:
                        rules.append((antecedent, consequent, confidence))
    return rules

def apriori(transactions,min_support, min_confidence):
    """Main Apriori algorithm."""
    total_transactions = len(transactions)
    transactions = [set(t) for t in transactions]  # ensure each transaction is a set

    # Step 1: Generate C1 (1-itemset candidates)
    item_counts = defaultdict(int)
    for transaction in transactions:
        for item in transaction:
            item_counts[frozenset([item])] += 1

    print(f"- Candidates of length 1 count: {len(item_counts)}")

    # Step 2: Filter C1 → L1 (frequent 1-itemsets)
    L1 = get_frequent_itemsets(transactions, item_counts.keys(), min_support, total_transactions)
    print(f"- After Pruning 1 count: {len(L1)}")
    support_data = {}
    support_data.update(L1)

    # Initialize the list of frequent itemsets
    L = [L1]

    k = 2
    while True:
        prev_L = list(L[-1].keys())  # Use the last frequent itemsets
        Ck = generate_new_candidates(prev_L, k)
        print(f"Candidates of length {k} count: {len(Ck)}")
        Lk = get_frequent_itemsets(transactions, Ck, min_support, total_transactions)
        print(f"After Pruning {k}-itemsets count: {len(Lk)}")

        if not Lk:
            break
        L.append(Lk)
        support_data.update(Lk)
        k += 1

    # Combine all frequent itemsets
    all_frequent_itemsets = {}
    for freq in L:
        all_frequent_itemsets.update(freq)

    # Generate rules
    rules = generate_association_rules(all_frequent_itemsets.keys(), min_confidence, support_data)
    print(f"\nTotal association rules generated: {len(rules)}")

    return rules

# Convert to DataFrame for easy grouping
df = pd.DataFrame(user_movie_data, columns=['user_id', 'movie_id'])

# Group by user and collect movie IDs into sets
transactions = df.groupby('user_id')['movie_id'].apply(set).tolist()
# Run the apriori algorithm
rules = apriori(transactions, 0.2, 0.8)

# Load the movie names to convert movie IDs into names
movies_df = pd.read_csv('movies.csv')
movie_id_to_name = dict(zip(movies_df['movieId'], movies_df['movie_name']))


- Candidates of length 1 count: 408
- After Pruning 1 count: 21
Candidates of length 2 count: 210
After Pruning 2-itemsets count: 36
Candidates of length 3 count: 55
After Pruning 3-itemsets count: 12
Candidates of length 4 count: 1
After Pruning 4-itemsets count: 0

Total association rules generated: 14


Finally, print your final association rules in the following format:

**movie_name_1, movie_name_2, ... --> movie_name_k**

where the movie names can be fetched by joining the movieId with the file 'movies.csv'. For example, one rule that you should find is:

**Jurassic Park (1993), Back to the Future (1985) --> Star Wars: Episode IV - A New Hope (1977)**

**Hint: You may need to use the Pandas library to load and process the movies.csv file, such as use pandas.read_csv() to load the data. https://pandas.pydata.org/pandas-docs/dev/user_guide/10min.html is a good place to learn the basics about Pandas.**

**Hint: if you implement the algorith correctly, you will find 14 rules in total:**
- Back to the Future (1985), Schindler's List (1993) -> Star Wars: Episode IV - A New Hope (1977)
- Godfather, The (1972), Godfather: Part II, The (1974) -> Star Wars: Episode IV - A New Hope (1977)
- Godfather: Part II, The (1974) -> Godfather, The (1972)
- Groundhog Day (1993), Princess Bride, The (1987) -> Back to the Future (1985)
- Groundhog Day (1993), Star Wars: Episode IV - A New Hope (1977) -> Back to the Future (1985)
- Jaws (1975) -> Star Wars: Episode IV - A New Hope (1977)
- Jurassic Park (1993), Back to the Future (1985) -> Star Wars: Episode IV - A New Hope (1977)
- Jurassic Park (1993), Princess Bride, The (1987) -> Star Wars: Episode IV - A New Hope (1977)
- Jurassic Park (1993), Saving Private Ryan (1998) -> Star Wars: Episode IV - A New Hope (1977)
- Princess Bride, The (1987), Back to the Future (1985) -> Star Wars: Episode IV - A New Hope (1977)
- Saving Private Ryan (1998), Back to the Future (1985) -> Star Wars: Episode IV - A New Hope (1977)
- Saving Private Ryan (1998), Princess Bride, The (1987) -> Star Wars: Episode IV - A New Hope (1977)
- Star Wars: Episode IV - A New Hope (1977), Godfather: Part II, The (1974) -> Godfather, The (1972)
- Star Wars: Episode IV - A New Hope (1977), Princess Bride, The (1987) -> Back to the Future (1985)

**Hint: the order of movies on the lefthand side does not matter, i.e., A,B->C is the same as B,A->C.**

In [21]:
# Write your code to print out the rules
for antecedent, consequent, confidence in rules:
    antecedent_movies = [movie_id_to_name[movie_id] for movie_id in antecedent]
    consequent_movies = [movie_id_to_name[movie_id] for movie_id in consequent]
    
    # Format the rule
    rule = f"{', '.join(antecedent_movies)} --> {', '.join(consequent_movies)}"
    print(rule)

Godfather: Part II, The (1974) --> Godfather, The (1972)
Jaws (1975) --> Star Wars: Episode IV - A New Hope (1977)
Saving Private Ryan (1998), Back to the Future (1985) --> Star Wars: Episode IV - A New Hope (1977)
Back to the Future (1985), Schindler's List (1993) --> Star Wars: Episode IV - A New Hope (1977)
Groundhog Day (1993), Princess Bride, The (1987) --> Back to the Future (1985)
Saving Private Ryan (1998), Princess Bride, The (1987) --> Star Wars: Episode IV - A New Hope (1977)
Star Wars: Episode IV - A New Hope (1977), Princess Bride, The (1987) --> Back to the Future (1985)
Princess Bride, The (1987), Back to the Future (1985) --> Star Wars: Episode IV - A New Hope (1977)
Groundhog Day (1993), Star Wars: Episode IV - A New Hope (1977) --> Back to the Future (1985)
Jurassic Park (1993), Back to the Future (1985) --> Star Wars: Episode IV - A New Hope (1977)
Jurassic Park (1993), Princess Bride, The (1987) --> Star Wars: Episode IV - A New Hope (1977)
Godfather, The (1972), Go